## 1. Primeros pasos con Spark

En este notebook vamos a ver una breve introducción a Spark con Python, y a una de sus abstracciones má comunes, los RDDs, conjuntos de datos distribuidos en un cluster. Para ello vamos a utilizar la librería PySpark, que es la API de Python para interactuar con la shell de Spark. Se puede instalar la librería mediante la siguiente guía:

- https://www.sicara.ai/blog/2017-05-02-get-started-pyspark-jupyter-notebook-3-minutes

### 1.1. SparkContext

El punto de entrada de cualquier programa Spark es un objeto SparkContext, este objeto le indica como conectarse a un clúster de Spark y crear RDDs. Para crer un SparkContext, primero es necesario cosntruir un objeto SparkConf conteniendo la información de la aplicación.

```Python
conf = SparkConf().setAppName('myApp').setMaster('local[*]')
sc = SparkContext(conf=conf)
```

El parámetro `local[*]` es una cadena especial que indica que está utilizando un clúster local. El asterisco * le indica a Spark que cree tantos hilos de trabajo como cores tenga la máquina.

In [1]:
# Importar librerías y dependencias
import findspark
findspark.init()

import pyspark
from pyspark import SparkContext, SparkConf

In [2]:
# SparkContext
conf = SparkConf().setAppName('myApp').setMaster('local[*]')
sc = SparkContext(conf=conf)

22/05/20 09:59:34 WARN Utils: Your hostname, gmachin resolves to a loopback address: 127.0.1.1; using 192.168.0.19 instead (on interface wlp3s0)
22/05/20 09:59:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/20 09:59:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# sc.setLogLevel('WARN')

Con este objeto, podemos acceder a la SparkUI:

In [3]:
sc

<SparkContext master=local[*] appName=myApp>

Y consultar el resto de configuraciones por defecto:

In [4]:
sc.getConf().getAll()

[('spark.app.id', 'local-1653033576696'),
 ('spark.driver.port', '36439'),
 ('spark.app.name', 'myApp'),
 ('spark.rdd.compress', 'True'),
 ('spark.driver.host', '192.168.0.19'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.app.startTime', '1653033575227')]

### 1.2. RDD

Spark gira en torno al concepto de *Resilient Distributed Dataset* (RDD), que es una colección de elementos distribuidos en el cluster y que se pueden operar en paralelo. Hay dos formas de crear un RDD: paralelizar una colección existente, o hacer referencia a un conjunto de datos en un sistema de almacenamiento externo, como un sistema de archivos compartido, HDFS, HBase o cualquier fuente de datos que ofrezca un formato de entrada Hadoop.

Para crear un RDD se utiliza el método `parallelize` del objeto SparkContext.

In [5]:
# Crear un RDD a partir de una colección de Python
rdd = sc.parallelize(range(10000000))
rdd

PythonRDD[1] at RDD at PythonRDD.scala:53

In [6]:
# Crear un RDD a partir de un fichero externo
rdd_quijote = sc.textFile("ficheros/extracto-quijote.txt")
rdd_quijote

ficheros/extracto-quijote.txt MapPartitionsRDD[3] at textFile at NativeMethodAccessorImpl.java:0

Un parámetro importante para los RDD es el número de particiones para distribuir el conjunto de datos. Spark ejecutará una task para cada partición del clúster. Por lo general, se utilizan de 2 a 4 particiones para cada CPU del clúster. Normalmente, Spark infiere la cantidad de particiones automáticamente en función de los recursos. Sin embargo, también puede configurarlo manualmente pasándolo como un segundo parámetro de parallelize.

In [7]:
# Ver las particiones de un RDD (valor 4 por defecto)
rdd.getNumPartitions()

4

In [8]:
rdd_quijote.getNumPartitions()

2

In [9]:
# Reparticionar un RDD
rdd_repartitioned = rdd.repartition(8)
rdd_repartitioned.getNumPartitions()

8

### 1.3. Operaciones con RDD

Spark permite dos tipos de operaciones sobre los RDD's:

- transformaciones
- acciones

Las transformaciones tienen como consecuencia la creación de un nuevo dataset a partir de uno ya existente, mientras que las acciones, devuelven un valor al driver del cluster después de realizar alguna operación sobre el dataset. La función `map()` por ejemplo sería una transformación, ya que aplica una función a cada valor de un dataset; por otro lado, la función `reduce()` devuelve un único valor fruto de algún tipo de agregación.

Las transformaciones en Spark se denominan *lazy*, ya que no se ejecutan hasta que se aplica una acción sobre el dataset. Esta característica permite que Spark se ejecute de una forma más eficiente. Por ejemplo si sobre un dataset realizamos un `map()` y luego un `reduce()`, al driver únicamente se le informará del valor de la acción y no de todos los pasos anteriores.

Algunas de las operaciones más usadas son:

- transformaciones: map, filter, reduceByKey
- acciones: reduce, collect, take, count

Las transformaciones y acciones definen lo que se conoce como un DAG (Directed Acyclic Graph), el cual represnta el job de Spark a ejecutar.

Aunque sc.parallelize y sc.textFile técnicamente no son transformaciones, podemos pensar en ellas como si lo fueran ya que son *lazy*. Si intentamos ver el contenido de lineLengths, veremos que no aparece nada, y solo da información del tipo de objeto que es, aún no se ha realizado ninguna operación.

In [15]:
# Ejemplo de operaciones con un RDD
# ------------------------------------------------------------------------------
# Definimos un puntero al fichero
lines = sc.textFile("ficheros/extracto-quijote.txt")

# Definimos un nuevo RDD sobre el puntero anterior
lineLengths = lines.map(lambda s: len(s))

lineLengths

PythonRDD[13] at RDD at PythonRDD.scala:53

Finalmente, realizamos una acción sobre el RDD lineLengths. Es en este punto, donde Spark comienza a ejecutar todo el trabajo sobre el RDD, para finalmente enviar un valor al driver del cluster. Las acciones son las que hacen que e motor de Spark genere el trabajo, y es cuando se pueden observas los jobs, stages y tasks en la SparkUI.

In [16]:
# Realizamos una acción sobre el RDD
totalLength = lineLengths.reduce(lambda a, b: a + b)

totalLength

176

Para demostrar de nuevo que la creación o transformación de un RDD son operaciones *lazy*, vamos a intentar visualizar los datos de un RDD, lo cual es imposible, a no ser que se utilice la acción `collect()`, la cual recoge todas las particiones del RDD y las convierte en una lista de Python.

Al intentar imprimir por pantalla el rdd lines, vemos que solo nos devuelve información de a donde apunta le objeto lines.

In [10]:
# Apuntar a un fichero de texto
lines = sc.textFile("ficheros/extracto-quijote.txt")
lines

ficheros/extracto-quijote.txt MapPartitionsRDD[10] at textFile at NativeMethodAccessorImpl.java:0

Al utilizar `collect()` sobre el RDD anterior, comprobamos que podemos imprimirlo por pantalla, y que se trata de una lista de Python.

In [11]:
# Mostrar el contenido del RDD
lines.collect()

['En un lugar de la Mancha, de cuyo nombre no quiero acordarme, ',
 'No ha mucho tiempo que vivía un hidalgo de los de lanza en',
 'astillero, adarga antigua, rocín flaco y galgo corredor.']

In [12]:
# Ver el tipo de objeto
type(lines.collect())

list

In [15]:
# Slice de una lista
lines.collect()[:1]

['En un lugar de la Mancha, de cuyo nombre no quiero acordarme, ']

Existen tres conceptos que están muy ligados a las operaciones que hemos visto en este punto, y que son los jobs, stages y tasks. Cada vez que se ejecuta una acción el motor de Spark ejecuta las operaciones programadas, dividiéndose estas en:

- Jobs: Trabajos de Spark que terminan en una acción
- Stages: Todas las parte de un job que se pueden ejecutar en una sola partición de Spark
- Tasks: Se realizan tantas Tasks como particiones haya en cada operación de un Stage

### 1.4. Persistencia de RDD

Como hemos visto en el punto anterior, las tranformaciones sobre un RDD son *lazy*, esto significa que Spark almacena como transformar cada RDD pero no ejecuta estas transformaciones hasta que se realiza una acción sobre uno de estos. Lo que en primera instancia podría ser una ventaja para evitar que la memoria de Spark llegue al límite, puede convertirse en un problema si tenemos que realizar repetidas acciones sobre un mismo RDD ya transformado, ya que acada vez que se quiera realizar una acción, deberán realizarse también todas las transformaciones anteriores, y cuando se trabaja con ficheros muy grandes, puede resultar costoso.

Para resolver este inconveniente, Spark cuenta con dos métodos, `persist()` y `cache()`, los cuales permiten almacenar en memoria un RDD sobre el que se vayan a realizar acciones de forma repetida.

In [24]:
from pyspark import StorageLevel

# Persist por defecto se realiza en memoria, pero pueden configurarse otras opciones
lineLengths.persist() # Equivale a lineLengths.persist(StorageLevel.ONLY_MEMORY)
lineLengths.persist(StorageLevel.MEMORY_AND_DISK)

PythonRDD[13] at RDD at PythonRDD.scala:53

In [25]:
lineLengths.unpersist()

PythonRDD[13] at RDD at PythonRDD.scala:53

## 2. Bibliografía

- https://spark.apache.org/docs/latest/rdd-programming-guide.html
- Libro: Introducción a Apache Spark 